# Morphing anatomies

Here we will have a look at how to morph anatomies within a common space, and how to morph them to an atlas. We will also have a look at `napari`, a useful tool for visualizing images and stacks within Python.

Before starting, make sure you pip install the following new libraries to the zenenv environment.

Close the notebook, shut down jupyter. From the same anaconda environment, run:
```
pip install napari[pyqt5]
pip install dipy
pip install bg-atlasapi
```

Now you should be ready to go!

In [ ]:
import tifffile
import napari
from matplotlib import Path
import numpy as np
from scipy.ndimage import zoom

# Load the data

First, load all the anatomies. The stacks have a resolution of `2.0×1.1×1.1 um`. After loading them, we will downsample to a final resolution of 3 um (in an real world data pipeline, you should probably try not to downsample)
To downsample, use the `zoom` function from `scipy.ndimage`:

In [ ]:
anatomies_folder = Path(r"...")
anatomy_files = sorted(anatomies_folder.glob("ana*"))

stack_res = (2, 1.1, 1.1)  # resolution of anatomy stacks
final_res = (3, 3, 3)  # target resolution of our resampling
anatomies = []
# Load the tiffs and use the zoom function to convert them to an isotropic resolution of (3, 3, 3) um
for single_file in anatomy_files:
    anatomy = tifffile.imread(str(single_file))
    anatomies.append(zoom(anatomy, (s / s_f for s, s_f in zip(stack_res, final_res))))

Let's have a look at them using napari!

In [ ]:
# We need to run this magi command for napari to run smoothly from jupyter notebook. 
# It is enough to call it one time in the notebook!
%gui qt

In [ ]:
v = napari.view_image(anatomies[0])
for anat in anatomies[1:]:
    v.add_image(anat)

# Registration

Finding the correct affine transformation for morphing a stack is an optimization problem: we need to find the best 12 entries for the affine transformation matrix (the last row is fixed, only zeros and final one) that maximise some similarity score (many could be used) between our transformed image and the reference we are morphing it to.

Since the parameter space is quite large, and many local minima can be found. Moreover, the data volume is very big. This is why we generally run multiple steps for the morphing procedure, where we first try a coarse registration of a binned/smoothed version of the stack, and then we tinker with the fine parameters. In the same way, we often first compute rigid transformations, and then we allow for deformations to be introduced for better fine matching. 

To morph data in a reference space, we will follow a 2 steps procedure:

1) First, we register all brains of the experiment onto the same representative brain from that experiment. In this way, we can look at all the experiment data within the same "space". For this, we will use an affine transformation.

2) Then, we will morph the average brain obtained in this "experiment space" onto the Max Planck Institute Zebrafish brain reference; here, we will use a non-affine (deforming) transformation.

For the registration, we will use the [dipy](https://dipy.org) library. Just for you to know, other options exist, such as [AntsPy](https://antspy.readthedocs.io/en/latest/) or [brainreg](https://github.com/brainglobe/brainreg).

## Within-experiment registration

### Register all brains

Here, we will run the affine registration to map all brains to the same space. For time reasons, you will have to run only one registration; we will provide you with the computed affine matrices for the others!

Registration procedures can be a bit of a pain, and there is a whole world to be explored behind them. If you want to have a more thorough overview of how dipy works, they have good [tutorials](https://dipy.org/documentation/1.3.0./examples_built/affine_registration_3d/#example-affine-registration-3d) on their functions. For this notebook, we will provide you most of the registration code below.

#### Define reference for internal registration

We need to arbitrarily define which onto which one of our fish brains we will  morph all the others. Here, we pick the first one; in general, just choose the one that is centered the best.

In [ ]:
reference = anatomies[0]

# Data-to-world transformation matrices. Dipy allow us to specify for each stack a "trasform_to_world" matrix
# where we would put pre-existing information about how the anatomical stacks relate to "real world",
# e.g. if they had different resolutions, or origin offsets. In our case, resolution is always the same, and 
# offsets are something that we need to figure out; so we'll leave it to identity matrices:
reference2world = np.eye(4)
moving2world = np.eye(4)

In [ ]:
from dipy.align.imaffine import MutualInformationMetric, AffineRegistration
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)

In [ ]:
# This defines a "metric" that can be used to give a number to "how similar 2 brains are". 
# There are multiple ways of doing this. Here, we are using mutual information:
nbins = 32
metric = MutualInformationMetric(nbins, None)


# As we said, we will use three different scales for calculating more and more precise transformations.
# For each one of those levels, we specify:

# 1) a sigma function that will blur the stack of some amount, to get rid of high resolution features:
sigmas = [10.0, 5.0, 2.0]

# 2) a downsampling factor, as for coarse transformations after blurring the stack 
# we won't miss much info if we downsample:
factors = [10, 5, 1]

# 3) A maximum number of iteration for the morphing algorithm to converge to an optimum:
level_iters = [1000, 100, 50]

# Finally, we put all this info in an object that keep together our parameters:
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

We will use 3 consecutive steps in the morphing:
1) First, we just find the best coarse traslation to match the two anatomies (**translation transformation**).

2) Second, we try to rotate the moving brain without deforming it, to improve the matching (**rigid transformation**).

3) Finally, we allow for the moving brain to be stretched to refine the matching (**affine transformation**).

The result of each step will be the starting point for the next transformation.
Note that each one of those 3 steps will run at three different spatial scales, as defined in the `AffineRegistration` object above!

This procedure can take some time. Here, ideally, it should be some minutes. For a full resolution stack and different registration parameters, it can easily get up to some hours!

In [ ]:
%%time

# The reference anatomy won't need to be morphed
anatomies_exp_space = [reference]

# Transformation matrix for the reference brain will be just identity matrix:
transf_matrices_to_exp = [np.eye(4)]

# Loop over all the anatomies but the first one that we used as reference:
for moving in anatomies[1:]:
    # Translation step:
    translation = affreg.optimize(reference, moving, TranslationTransform3D(), None,
                                      reference2world, moving2world
                                      )

    # Rigid transformation step:
    rigid = affreg.optimize(reference, moving, RigidTransform3D(), None,
                                      reference2world, moving2world,
                                      starting_affine=translation.affine)

    # Affine transformation step:
    affine = affreg.optimize(reference, moving, AffineTransform3D(), None,
                                      reference2world, moving2world,
                                      starting_affine=rigid.affine)


    # Calculate morphed stack, and append to list:
    anatomies_exp_space.append(affine.transform(moving))

    # Save the transformation matrix (for morphing the coordinates we need the "affine_inv" 
    # attribute of the final transformation object):
    transf_matrices_to_exp.append(affine.affine_inv)

Now use napari to have a look at the morphed stacks one by one! We colored them differently and use additive blending to have them overlapping in the viewer

In [ ]:
v = napari.view_image(anatomies_exp_space[0], colormap="green")
v.add_image(anatomies_exp_space[1], colormap="magenta", blending="additive")

The results won't be perfect. For your real world data, there generally has to be some tinkering with the transformation parameters; importantly, often we would use a non-affine transformation to improve the transform (that can warp the stack, instead of just shearing it).

### Compute average anatomy

Now, we average all the anatomies togheter, to create a general anatomy for the experiment where fish to fish variability is averaged out:

In [ ]:
super_anatomy = np.nanmean(np.stack(anatomies_exp_space), 0)

In [ ]:
napari.view_image(super_anatomy)

# Morph to a brain reference using BrainGlobe

Now, we will morph our super anatomy onto the [Max Planck Institute](https://fishatlas.neuro.mpg.de) zebrafish atlas. In this way, we can have our data (eg, ROI locations) in a space that contains annotations for all regions of the brain, and we can look at activity within different brain regions. Moreover, morph data on common reference spaces facilitates a lot sharing of the data between different labs!

To download the reference brain, we will rely on the [BrainGlobe atlas API](https://github.com/brainglobe/bg-atlasapi).

### Download and downsample the reference stack

In [ ]:
from bg_atlasapi.bg_atlas import BrainGlobeAtlas

# This will automatically trigger the download of the zebrafish atlas. This atlas has a resolution of 1 um
atlas = BrainGlobeAtlas("mpin_zfish_1um")

In [ ]:
# To make the registration faster, we downsample also the atlas to the final resolution of 3 um:

atlas_down = zoom(atlas.reference, (s/f_s for s, f_s in zip(atlas.resolution, final_res)))

## Match reference and anatomy orientation

Before starting the transformation, we ned to make sure that the anatomical stacks have the same orientation.
There are many ways of doing this; it is usually an annoying procedure that requires a lot of arbitrary dimension swapping and flipping. To facilitate things, we will use the [bg-space](https://github.com/brainglobe/bg-space) package. Have a look at the readme there before continuing!

In [ ]:
# Use the AnatomicalSpace class to map the data onto the Brainglobe anatomical space. 
# Try to figure out the orientation from the bg-space tutorial! 
# The origin location for the reference can be found in atlas.space.origin; 
# the resolution is 3 um after downsampling.

# Use napari to check if the two stacks have the same orientation! (they won't be aligned yet of course)
from bg_space import AnatomicalSpace 

sc_atlas = AnatomicalSpace("asr", resolution=final_res) 
sc_mydata = AnatomicalSpace("slp", resolution=final_res, shape=super_anatomy.shape)

# Use the map_stack_to method:
mydata_bg_space = sc_mydata.map_stack_to(sc_atlas, super_anatomy)
# mydata_bg_space_z = mydata_bg_space / 50

v = napari.view_image(atlas_down, colormap="green")
v.add_image(mydata_bg_space, colormap="magenta", blending="additive")

#### Affine transformation
If, like in this case, the stacks are in very different spaces, before starting the actual affine transformation it is useful to run a first simple alignment that just align the two stacks using their center of mass.

Then, we calculate the affine transformation as we did before:

In [ ]:
from dipy.align.imaffine import transform_centers_of_mass

In [ ]:
%%time

# Additional step: center of mass
c_of_mass = transform_centers_of_mass(atlas_down, reference2world,
                                      mydata_bg_space, moving2world)

# Translation: 
translation = affreg.optimize(atlas_down, mydata_bg_space, TranslationTransform3D(), None,
                              reference2world, moving2world,
                             starting_affine=c_of_mass.affine)

# Rigid transformation:
rigid = affreg.optimize(atlas_down, mydata_bg_space, RigidTransform3D(), None,
                        reference2world, moving2world,
                        starting_affine=translation.affine)

# Affine transformation. Could be needed, does not change the results much here, and takes long time:
# affine = affreg.optimize(atlas_down, mydata_bg_space, AffineTransform3D(), None,
#                          reference2world, moving2world,
#                         starting_affine=rigid.affine)

# Finally, transform the stack:
morphed_super_anatomy = rigid.transform(mydata_bg_space)

In [ ]:
# Display the results using napari:

v = napari.view_image(morphed_super_anatomy, colormap="green")
v.add_image(atlas_down, colormap="magenta", blending="additive")

#### Non affine transformation

To improve the quality of the morphing to the atlas reference space, we could add after the affine transformation a step where we use an algorithm that can deform locally the stack to make it matching better the reference brain (see [here](https://dipy.org/documentation/1.3.0./examples_built/syn_registration_2d/#example-syn-registration-2d) for an introduction).

We won't do it here, for time purposes and because dipy does not allow us to use a non-affine transformation to convert coordinates of extracted ROIs. In the lab, we use AntsPy for this purpose.

### Final morphing of anatomies in reference space

The final step is now to bring the anatomy stacks that we loaded at the beginning of the notebook all the way into the atlas reference space. For doing this, we won't need to calculate any further registration! We already have the transformation from each stack to the "experiment reference", and from the experiment reference to the atlas space. We just need to transform the stacks that we already morphed in the experiment space. Remember, we need to include all the steps that we followed, including the reorientations!

In [ ]:
%%time
anatomies_atlas_space = []
for anatomy_exp_space in anatomies_exp_space:
    # Reorient using bg_space:
    reoriented = sc_mydata.map_stack_to(sc_atlas, anatomy_exp_space)
    
    # Affine transformation to atlas space:
    affine_trasf = rigid.transform(reoriented)
    
    # Nonaffine transform:
    # anatomy_atlas_space = nonaffine_mapping.transform(affine_trasf)
    
    anatomies_atlas_space.append(affine_trasf)

In [ ]:
v = napari.view_image(atlas_down, colormap="gray")
v.add_image(anatomies_atlas_space[0], colormap="magenta", blending="additive")
v.add_image(anatomies_atlas_space[1], colormap="blue", blending="additive")
v.add_image(anatomies_atlas_space[2], colormap="yellow", blending="additive")
v.add_image(anatomies_atlas_space[3], colormap="green", blending="additive")

# Morph coordinates in reference space

After we have found a suitable transformation, we can apply it to coordinates as well. dipy does not allow for morphing coordinates with a non-affine transformations, while other libraries such as antspy provide that function as well.

Therefore, we will only morph the coordinates using the affine transformation that we got.

Together with the anatomies, you should have downloaded npy files with the coordinates of ROIs detected in each one of the anatomical stacks.

In [ ]:
import pandas as pd

# Load the coordinates:
original_coords = [np.load(f) for f in sorted(anatomies_folder.glob("*.npy"))]

To morph the coordinates we need to make sure we include all steps that we used to go from the original stacks to the stacks:

### Follow coordinate transformation step by step:

In [ ]:
i = 1
coords = original_coords[i].copy()
affine_to_exp = transf_matrices_to_exp[i]
anat_exp_space = anatomies_exp_space[i]
anatomy = anatomies[i]
anat_atlas_space = anatomies_atlas_space[i]


# match stack size 
for i in range(3):
    coords[i, :] = coords[i, :] * (stack_res[i] / final_res[i])
    
coords = np.concatenate([coords, np.ones((1, coords.shape[1]))], axis=0)

In [ ]:
# Original space:
v = napari.view_image(anatomy)
v.add_points(coords[:3, :].T, size=2, n_dimensional=True)

In [ ]:
# Experiment space:
v = napari.view_image(anat_exp_space)
v.add_points((affine_to_exp @ coords)[:3, :].T, size=2, n_dimensional=True)

In [ ]:
# Moved to atlas orientation:
aff_to_atlas_sp = sc_mydata.transformation_matrix_to(sc_atlas)
v = napari.view_image(sc_mydata.map_stack_to(sc_atlas, anat_exp_space))
v.add_points((aff_to_atlas_sp @ affine_to_exp @ coords)[:3, :].T, size=2, n_dimensional=True)

In [ ]:
# In atlas space
v = napari.view_image(atlas_down, colormap="green")
v.add_image(anat_atlas_space, colormap="magenta", blending="additive")
v.add_points((rigid.affine_inv @ aff_to_atlas_sp @ affine_to_exp @ coords)[:3, :].T, 
             size=2, n_dimensional=True)

### Transform coordinates for all fish:

In [ ]:
# Now the code you just saw to transform in a loop the coordinates from all 4 experiments and visualize them with Napari!
morphed_coords = []
for i, (affine_to_exp, coords) in enumerate(zip(transf_matrices_to_exp, original_coords)):
    
    # match stack size and append ones:
    for i in range(3):
        coords[i, :] = coords[i, :] * (stack_res[i] / final_res[i])

    coords = np.concatenate([coords, np.ones((1, coords.shape[1]))], axis=0)


    coords_atlas_space = (rigid.affine_inv @ aff_to_atlas_sp @ affine_to_exp @ coords)[:3, :].T
    morphed_coords.append(coords_atlas_space)
    
    # save
    np.save(anatomies_folder / f"morphed_coords{i}.npy")

In [ ]:
v = napari.view_image(anat_atlas_space, colormap="gray", blending="additive")
v.add_points(morphed_coords[0], 
             size=2, n_dimensional=True, face_color="red")
v.add_points(morphed_coords[1], 
             size=2, n_dimensional=True, face_color="blue")
v.add_points(morphed_coords[2], 
             size=2, n_dimensional=True, face_color="green")
v.add_points(morphed_coords[3], 
             size=2, n_dimensional=True, face_color="yellow")